In [1]:
import numpy as np

#from scipy import stats

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 14

## Anemometro a filo caldo

Analisi dati e determinazione **frequenza di Strouhal**

## Lettura dati

In [ ]:
data = np.loadtxt('./')